In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
df = pd.read_csv('government_data.csv')
df_copy = df.copy()
df_copy.head(3)

,發票年月,縣市代碼,縣市名稱,鄉鎮市區代碼,鄉鎮市區名稱,行業名稱,平均開立張數,平均開立金額,平均客單價
0,2021/9/1,A,臺北市,10.0,中山區,百貨公司,149982,225759520,1505
1,2021/9/1,A,臺北市,2.0,大安區,百貨公司,71125,164283711,2310
2,2021/9/1,A,臺北市,17.0,信義區,百貨公司,100730,150002101,1489


In [3]:
df_copy.dtypes

發票年月       object
縣市代碼       object
縣市名稱       object
鄉鎮市區代碼    float64
鄉鎮市區名稱     object
行業名稱       object
平均開立張數      int64
平均開立金額      int64
平均客單價       int64
dtype: object

In [4]:
df_copy['發票年月'] = pd.to_datetime(df_copy['發票年月'])

In [5]:
display(df_copy.head(3))
df_copy.dtypes

,發票年月,縣市代碼,縣市名稱,鄉鎮市區代碼,鄉鎮市區名稱,行業名稱,平均開立張數,平均開立金額,平均客單價
0,2021-09-01,A,臺北市,10.0,中山區,百貨公司,149982,225759520,1505
1,2021-09-01,A,臺北市,2.0,大安區,百貨公司,71125,164283711,2310
2,2021-09-01,A,臺北市,17.0,信義區,百貨公司,100730,150002101,1489


發票年月      datetime64[ns]
縣市代碼              object
縣市名稱              object
鄉鎮市區代碼           float64
鄉鎮市區名稱            object
行業名稱              object
平均開立張數             int64
平均開立金額             int64
平均客單價              int64
dtype: object

In [6]:
df_1819 = df_copy[df_copy['發票年月'] < pd.Timestamp(year=2020, month=1, day=1)]
df_1819.head(3)

,發票年月,縣市代碼,縣市名稱,鄉鎮市區代碼,鄉鎮市區名稱,行業名稱,平均開立張數,平均開立金額,平均客單價
3241,2019-12-01,A,臺北市,3.0,中正區,百貨公司,153663,204143961,1329
3242,2019-12-01,T,屏東縣,1.0,屏東市,百貨公司,51017,37092722,727
3243,2019-12-01,E,高雄市,67.0,燕巢區,綜合商品零售業,40426,3949934,98


In [7]:
df_copy.loc[list(df_1819.index), 'COVID'] = 0
df_copy['COVID'] = df_copy['COVID'].fillna(1)
df_copy.head(3)

,發票年月,縣市代碼,縣市名稱,鄉鎮市區代碼,鄉鎮市區名稱,行業名稱,平均開立張數,平均開立金額,平均客單價,COVID
0,2021-09-01,A,臺北市,10.0,中山區,百貨公司,149982,225759520,1505,1.0
1,2021-09-01,A,臺北市,2.0,大安區,百貨公司,71125,164283711,2310,1.0
2,2021-09-01,A,臺北市,17.0,信義區,百貨公司,100730,150002101,1489,1.0


In [8]:
df_copy['Year'] = df_copy['發票年月'].apply(lambda x: x.year)
df_copy['Month'] = df_copy['發票年月'].apply(lambda x: x.month)
df_copy['Day'] = df_copy['發票年月'].apply(lambda x: x.day)

In [9]:
df_anniversary = df_copy[(df_copy['Month'] <= 11) & (df_copy['Month'] >= 10)]
df_anniversary.Month.value_counts()

11    2880
10    2867
Name: Month, dtype: int64

In [10]:
df_copy.loc[list(df_anniversary.index), 'Anniversary'] = 1
df_copy['Anniversary'] = df_copy['Anniversary'].fillna(0)

In [11]:
df_nearanni = df_copy[(df_copy['Month'] == 9)]
df_copy.loc[list(df_anniversary.index), '週年慶前後'] = -1
df_copy.loc[list(df_nearanni.index), '週年慶前後'] = 1
df_copy['週年慶前後'].fillna(0, inplace=True)

In [12]:
df_copy.head(3)

,發票年月,縣市代碼,縣市名稱,鄉鎮市區代碼,鄉鎮市區名稱,行業名稱,平均開立張數,平均開立金額,平均客單價,COVID,Year,Month,Day,Anniversary,週年慶前後
0,2021-09-01,A,臺北市,10.0,中山區,百貨公司,149982,225759520,1505,1.0,2021,9,1,0.0,1.0
1,2021-09-01,A,臺北市,2.0,大安區,百貨公司,71125,164283711,2310,1.0,2021,9,1,0.0,1.0
2,2021-09-01,A,臺北市,17.0,信義區,百貨公司,100730,150002101,1489,1.0,2021,9,1,0.0,1.0


In [14]:
dep_dist_list = list(df_copy[df_copy['行業名稱'] == '百貨公司'].鄉鎮市區名稱.unique())

In [15]:
dist_data_index = []  # 區內有百貨
not_dist_data_index = []  # 區內沒有百貨

for index, row in df_copy.iterrows():
    if row['鄉鎮市區名稱'] in dep_dist_list and (row['Year'] >= 2018 and row['Year'] <= 2020):
        dist_data_index.append(index)
    elif row['鄉鎮市區名稱'] not in dep_dist_list and (row['Year'] >= 2018 and row['Year'] <= 2020):
        not_dist_data_index.append(index)

In [16]:
df_copy.loc[dist_data_index, '區內有百貨'] = 1
df_copy.loc[not_dist_data_index, '區內有百貨'] = 0
df_copy['區內有百貨'].fillna(-1, inplace=True)

In [18]:
df_copy['區內有百貨'].value_counts()

 0.0    28752
-1.0     9177
 1.0     5223
Name: 區內有百貨, dtype: int64

In [19]:
df_copy.to_excel('government_data_2.0.xlsx', index=False)

In [17]:
# df_dep_dist.to_excel('百貨公司所在區.xlsx', index=False)